In [1]:
!pip install yfinance mplfinance timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.2 MB/s eta 0:00:00


In [2]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
from sklearn import model_selection
from PIL import Image
from torchvision import transforms
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import timm
import torch.nn as nn
from torchvision import models
import torch.optim as optim
from torch.utils.data import DataLoader
import time
import copy
from datetime import datetime, timedelta
import os

In [3]:


data = yf.download('EURJPY=X', start='2020-01-03', end='2024-01-21')

data.head()


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-03,121.239998,121.277000,120.175003,121.245003,121.245003,0
2020-01-06,120.542999,121.265999,120.474998,120.500999,120.500999,0
2020-01-07,121.367996,121.440002,120.787003,121.360001,121.360001,0
2020-01-08,120.434998,121.237000,120.175003,120.485001,120.485001,0
2020-01-09,121.203003,121.675003,121.160004,121.168999,121.168999,0


In [4]:
data['Up'] = 0
# "future_num" days after the close predicted up or down
future_num = 1

for i in range(len(data)-future_num):
    if data['Close'][i] < data['Close'][i+future_num]:
        data['Up'][i] = 1
"""
data['Up'] = 0
data['Up'][data['Close'] < data['Close'].shift(-future_num)] = 1
"""

<ipython-input-4-164d733baf9e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Up'][i] = 1


"\ndata['Up'] = 0\ndata['Up'][data['Close'] < data['Close'].shift(-future_num)] = 1\n"

In [5]:
data.head()

,Open,High,Low,Close,Adj Close,Volume,Up
Date,,,,,,,
2020-01-03,121.239998,121.277000,120.175003,121.245003,121.245003,0,0
2020-01-06,120.542999,121.265999,120.474998,120.500999,120.500999,0,1
2020-01-07,121.367996,121.440002,120.787003,121.360001,121.360001,0,0
2020-01-08,120.434998,121.237000,120.175003,120.485001,120.485001,0,1
2020-01-09,121.203003,121.675003,121.160004,121.168999,121.168999,0,1


In [6]:
# Set path to save train images
folder_path = './DataFolder'

# If the directory does not exist, create it
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"The directory '{folder_path}' has been created!")
else:
    print(f"The directory '{folder_path}' has already exist!")

ディレクトリ './DataFolder' が作成されました。


In [7]:
data['ImageName'] =  [folder_path +'/'+ str(data.index[i].year)+ '-'  + str(data.index[i].month) + '-' + str(data.index[i].day) + '.png' for i in range(len(data))]

In [8]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume,Up,ImageName
Date,,,,,,,,
2024-01-15,158.865005,159.824005,158.822998,158.863998,158.863998,0,1,./DataFolder/2024-1-15.png
2024-01-16,159.563004,160.000000,159.242996,159.563004,159.563004,0,1,./DataFolder/2024-1-16.png
2024-01-17,160.179993,161.214996,160.018005,160.179993,160.179993,0,1,./DataFolder/2024-1-17.png
2024-01-18,161.223007,161.367004,160.656006,161.223007,161.223007,0,0,./DataFolder/2024-1-18.png
2024-01-19,161.126999,161.839005,160.807007,161.126999,161.126999,0,0,./DataFolder/2024-1-19.png


In [11]:
from tqdm import tqdm
# Moving Average Calculation
short_window = 5   # 5-day moving average
long_window = 20   # 20-day moving average
data['SMA5'] = data['Close'].rolling(window=short_window, min_periods=1).mean()
data['SMA20'] = data['Close'].rolling(window=long_window, min_periods=1).mean()

# Chart Style Setting
mc = mpf.make_marketcolors(up='#77d879', down='#db3f3f', inherit=True)
s = mpf.make_mpf_style(base_mpf_style='nightclouds', marketcolors=mc, gridstyle='')

figsize = 2.56, 2.56
# Generate Charts
seq_len = 20
for i in tqdm(range(len(data) - seq_len + 1)):
    c = data.iloc[i:i + seq_len]

    # Additional plots of moving average line
    add_plot = [
        mpf.make_addplot(c['SMA5'], color='blue', alpha=0.5),
        mpf.make_addplot(c['SMA20'], color='red', alpha=0.5),
    ]
    """
    fig, ax = mpf.plot(c, type='candle', style=s, addplot=add_plot, returnfig=True)
    # Axis setting
    ax[0].set_xticklabels([])
    ax[0].set_yticklabels([])
    ax[0].xaxis.set_visible(False)
    ax[0].yaxis.set_visible(False)
    """
    fig, ax = mpf.plot(c, type='candle', style=s, addplot=add_plot, figsize=figsize, returnfig=True)

    # Axis setting
    ax[0].set_xticklabels([])
    ax[0].set_yticklabels([])
    ax[0].xaxis.set_visible(False)
    ax[0].yaxis.set_visible(False)

    # Save file as an image
    pngfile = data['ImageName'][i + seq_len - 1]
    fig.savefig(pngfile, pad_inches=0.1, transparent=False,bbox_inches='tight')
    plt.close(fig)

print("Candlestick charts and moving averages have been successfully generated!")

100%|██████████| 1037/1037 [02:17<00:00,  7.52it/s]

キャンドルスティックチャートと移動平均線の生成が完了しました。


In [12]:
data

,Open,High,Low,Close,Adj Close,Volume,Up,ImageName,SMA10,SMA50,SMA5,SMA20
Date,,,,,,,,,,,,
2020-01-03,121.239998,121.277000,120.175003,121.245003,121.245003,0,0,./DataFolder/2020-1-3.png,121.245003,121.245003,121.245003,121.245003
2020-01-06,120.542999,121.265999,120.474998,120.500999,120.500999,0,1,./DataFolder/2020-1-6.png,120.873001,120.873001,120.873001,120.873001
2020-01-07,121.367996,121.440002,120.787003,121.360001,121.360001,0,0,./DataFolder/2020-1-7.png,121.035334,121.035334,121.035334,121.035334
2020-01-08,120.434998,121.237000,120.175003,120.485001,120.485001,0,1,./DataFolder/2020-1-8.png,120.897751,120.897751,120.897751,120.897751
2020-01-09,121.203003,121.675003,121.160004,121.168999,121.168999,0,1,./DataFolder/2020-1-9.png,120.952000,120.952000,120.952000,120.952000
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-15,158.865005,159.824005,158.822998,158.863998,158.863998,0,1,./DataFolder/2024-1-15.png,157.822800,159.182307,158.778201,157.286417
2024-01-16,159.563004,160.000000,159.242996,159.563004,159.563004,0,1,./DataFolder/2024-1-16.png,158.202600,159.158466,159.104001,157.469917
2024-01-17,160.179993,161.214996,160.018005,160.179993,160.179993,0,1,./DataFolder/2024-1-17.png,158.667000,159.145026,159.562799,157.579066


In [13]:
import numpy as np
#x_train, x_val, y_train, y_val = model_selection.train_test_split(data['ImageName'].values, data['Up'].values, test_size=0.1,random_state=1)
num_val = 10
val_start_index = len(data) - num_val

# Split into training and validation data
x_train = data['ImageName'].values[:val_start_index]
y_train = data['Up'].values[:val_start_index]
z_train = np.array(data.index[:val_start_index].strftime('%Y-%m-%d').tolist())
x_val = data['ImageName'].values[val_start_index:]
y_val = data['Up'].values[val_start_index:]
z_val = np.array(data.index[val_start_index:].strftime('%Y-%m-%d').tolist())


In [14]:
z_val

array(['2024-01-08', '2024-01-09', '2024-01-10', '2024-01-11',
       '2024-01-12', '2024-01-15', '2024-01-16', '2024-01-17',
       '2024-01-18', '2024-01-19'], dtype='<U10')

In [15]:
y_val

array([0, 0, 1, 0, 0, 1, 1, 1, 0, 0])

In [16]:
X = []
Y = []
Z = []
X_val = []
Y_val = []
Z_val = []
transform = transforms.Compose([transforms.ToTensor()])

#for image_file,label in zip(x_train,y_train):
for image_file,label,date in zip(x_train,y_train,z_train):
    try:
        image = Image.open(image_file)
        image = image.convert('RGB')
        image = transform(image)
        #image = np.asarray(image)
        X.append(image)
        Y.append(label)
        Z.append(date)
    except FileNotFoundError as e:
        print(e)


for image_file,label,date in zip(x_val,y_val,z_val):
    try:
        image = Image.open(image_file)
        image = image.convert('RGB')
        image = transform(image)
        X_val.append(image)
        Y_val.append(label)
        Z_val.append(date)
    except FileNotFoundError as e:
        print(e)

[Errno 2] No such file or directory: './DataFolder/2020-1-3.png'
[Errno 2] No such file or directory: './DataFolder/2020-1-6.png'
[Errno 2] No such file or directory: './DataFolder/2020-1-7.png'
[Errno 2] No such file or directory: './DataFolder/2020-1-8.png'
[Errno 2] No such file or directory: './DataFolder/2020-1-9.png'
[Errno 2] No such file or directory: './DataFolder/2020-1-10.png'
[Errno 2] No such file or directory: './DataFolder/2020-1-13.png'
[Errno 2] No such file or directory: './DataFolder/2020-1-14.png'
[Errno 2] No such file or directory: './DataFolder/2020-1-15.png'
[Errno 2] No such file or directory: './DataFolder/2020-1-16.png'
[Errno 2] No such file or directory: './DataFolder/2020-1-17.png'
[Errno 2] No such file or directory: './DataFolder/2020-1-20.png'
[Errno 2] No such file or directory: './DataFolder/2020-1-21.png'
[Errno 2] No such file or directory: './DataFolder/2020-1-22.png'
[Errno 2] No such file or directory: './DataFolder/2020-1-23.png'
[Errno 2] No su

In [17]:
import numpy as np
# Set directory path
folder_path = './data'

# If the directory does not exist, create it.
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"The directory '{folder_path}' has been created!")
else:
    print(f"The directory '{folder_path}' has already exist!")

# Save as numpy array
np.save('./data/x_train.npy', x_train)
np.save('./data/x_val.npy', x_val)
np.save('./data/y_train.npy', y_train)
np.save('./data/y_val.npy', y_val)
np.save('./data/z_train.npy', z_train)
np.save('./data/z_val.npy', z_val)

ディレクトリ './data' が作成されました。


In [18]:
class EUR_Dataset(Dataset):
    def __init__(self, image, label_list,date_list, phase=None):

        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(256)
        ])
        self.image = image
        self.label_list = torch.tensor(label_list,dtype=torch.long)
        self.phase = phase
        self.date_list = date_list

    def __getitem__(self,index):
        # Load and preprocess the index-th image
        image = self.image[index]
        # Get the index-th label
        label = self.label_list[index]
        date = self.date_list[index]
        return image, label, date

    def __len__(self):
        return len(self.image)

In [19]:
train_dataset = EUR_Dataset(X,Y,Z,phase='train')
total_samples = len(train_dataset)
print(total_samples)
val_dataset = EUR_Dataset(X_val,Y_val,Z_val,phase='val')
val_samples = len(val_dataset)
print(val_samples)

1027
10


In [20]:

model = models.resnet18(pretrained=True)
#model = models.resnet152(pretrained=True)
torch.cuda.empty_cache()
num_ftrs = model.fc.in_features

model.fc  = nn.Sequential(
                      nn.Linear(num_ftrs ,2))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 159MB/s]


In [21]:
from torch.optim import lr_scheduler

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

criterion = nn.CrossEntropyLoss()


# Observe that all parameters are being optimized
optimizer_ft = optim.AdamW(model.parameters())  # Initial learning rate set to 0.01
num_epochs = 40
# Defining LambdaLR


batchsize = 64

loss_list = []
val_loss_list = []
# Create instance of DataLoaders
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batchsize, shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=batchsize, shuffle=False)

# Store data loader as a dictionary
dataloaders_dict = {
    'train' : train_dataloader,
    'val' : val_dataloader
}

cuda:0


In [23]:
Train_loss = []
Val_loss = []
Train_acc = []
Val_acc = []


def train_model(model, criterion, optimizer,  num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    #best_acc = 0.0
    best_loss = 1000.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        lambda1 = lambda epoch: 0.1 if epoch >= num_epochs // 2 else 1.0  # num_epochs is the total number of epochs
        scheduler = lr_scheduler.LambdaLR(optimizer_ft, lr_lambda=lambda1)
        # Each epoch has a training and validation phase

        for phase in ['train', 'val']:
            running_loss = 0.0
            running_corrects = 0

            # Switching between learning mode and validation mode
            if phase == 'train':
                model.train()
            else:
                model.eval()

            #for i,data in enumerate(trainloader):
            i = 0
            data_num = 0
            for inputs, labels, dates in dataloaders_dict[phase]:
                i += 1
                #for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    preds = torch.argmax(outputs, axis=1)

                    #print(preds)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step()
                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                    data_num += inputs.size(0)

            epoch_loss = running_loss/data_num
            epoch_acc = running_corrects.double()/data_num


            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'train':
                Train_loss.append(epoch_loss)
                Train_acc.append(epoch_acc)
            else:
                Val_loss.append(epoch_loss)
                Val_acc.append(epoch_acc)

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [24]:
model = train_model(model,criterion,optimizer_ft,num_epochs=num_epochs)

Epoch 0/39
----------
train Loss: 0.8737 Acc: 0.4995
val Loss: 2.4197 Acc: 0.4000

Epoch 1/39
----------
train Loss: 0.7170 Acc: 0.4985
val Loss: 1.0689 Acc: 0.4000

Epoch 2/39
----------
train Loss: 0.6397 Acc: 0.6368
val Loss: 1.0974 Acc: 0.4000

Epoch 3/39
----------
train Loss: 0.5836 Acc: 0.7030
val Loss: 0.5715 Acc: 0.8000

Epoch 4/39
----------
train Loss: 0.4739 Acc: 0.7780
val Loss: 0.8593 Acc: 0.6000

Epoch 5/39
----------
train Loss: 0.4426 Acc: 0.7994
val Loss: 0.6155 Acc: 0.8000

Epoch 6/39
----------
train Loss: 0.3787 Acc: 0.8354
val Loss: 0.8664 Acc: 0.4000

Epoch 7/39
----------
train Loss: 0.3337 Acc: 0.8656
val Loss: 0.7144 Acc: 0.8000

Epoch 8/39
----------
train Loss: 0.2594 Acc: 0.8929
val Loss: 0.6213 Acc: 0.6000

Epoch 9/39
----------
train Loss: 0.2084 Acc: 0.9133
val Loss: 0.3203 Acc: 0.8000

Epoch 10/39
----------
train Loss: 0.3810 Acc: 0.8481
val Loss: 0.2704 Acc: 0.9000

Epoch 11/39
----------
train Loss: 0.2966 Acc: 0.8754
val Loss: 0.3857 Acc: 0.8000

Ep

In [25]:
# Record prediction results and prediction probabilities
val_dataloader = DataLoader(dataset=val_dataset, batch_size=1, shuffle=False)
pred_list = np.empty([1, 2])

i = 0
input_list = []
output_list = []
sample_list = []
for inputs, labels,dates in val_dataloader:
    i += 1
    #for inputs, labels in dataloaders[phase]:
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(inputs)
        sm = nn.Softmax(dim=1)
        outputs2 = np.array(sm(outputs.cpu()))
        pred_list = np.vstack((pred_list,outputs2))
        output_list.append(outputs2)
        input_list.append(labels)
        labels_np = labels.cpu().numpy()  # Convert tensor to NumPy array
        sample_list.append([labels_np,outputs2,dates])
print(sample_list)

[[array([0]), array([[0.9863154 , 0.01368457]], dtype=float32), ('2024-01-08',)], [array([0]), array([[0.99196386, 0.00803618]], dtype=float32), ('2024-01-09',)], [array([1]), array([[0.01213066, 0.9878693 ]], dtype=float32), ('2024-01-10',)], [array([0]), array([[0.9356042, 0.0643957]], dtype=float32), ('2024-01-11',)], [array([0]), array([[0.94881475, 0.05118523]], dtype=float32), ('2024-01-12',)], [array([1]), array([[0.00296727, 0.99703276]], dtype=float32), ('2024-01-15',)], [array([1]), array([[0.12403503, 0.87596494]], dtype=float32), ('2024-01-16',)], [array([1]), array([[0.00285717, 0.99714285]], dtype=float32), ('2024-01-17',)], [array([0]), array([[0.511524  , 0.48847598]], dtype=float32), ('2024-01-18',)], [array([0]), array([[0.5640743 , 0.43592572]], dtype=float32), ('2024-01-19',)]]


In [34]:
data = {'Label': [], 'Pred_down': [], 'Pred_up': [], 'Date': []}

# Analyze each element of sample_list and add it to the data list
for item in sample_list:
    data['Label'].append(item[0][0])
    data['Pred_down'].append(item[1][0][0])
    data['Pred_up'].append(item[1][0][1])
    data['Date'].append(item[2][0])

# Converted to Pandas data frame
df = pd.DataFrame(data)

In [41]:
def print_ext(updown, rate, data):
    UPtarget = 1 if updown == 'UP' else 0

    # Extract rows from the data frame that match the condition (Pred_zero or Pred_one exceeds the threshold)
    filtered_data = data[((data['Pred_down'] > rate) | (data['Pred_up'] > rate)) & (data['Label'] == UPtarget)]

    if UPtarget == 1:
      target = filtered_data['Pred_up']
    else:
      target = filtered_data['Pred_down']
    # Number of rows matching the condition that were correctly predicted
    correct_predictions = filtered_data[(target > rate)]

    counter = len(correct_predictions)

    total_counter = len(filtered_data)
    print(filtered_data)
    # Display the result
    if total_counter > 0:
        accuracy = counter / total_counter
        print('Accuracy: {:.4f}'.format(accuracy))
    else:
        print('Accuracy: N/A - division by zero')
    print('Number of correct answers within the predicted: {}'.format(counter))
    print('Number of {} exceeding the threshold {}: {}'.format(updown,rate, total_counter))
    print("--------------------------------------------------------------")



In [42]:
print_ext('UP', 0.5, df)
print_ext('DOWN', 0.5, df)
print_ext('UP', 0.7, df)
print_ext('DOWN', 0.7, df)
print_ext('UP', 0.8, df)
print_ext('DOWN', 0.8, df)
print_ext('UP', 0.9, df)
print_ext('DOWN', 0.9, df)

   Label  Pred_down   Pred_up        Date
2      1   0.012131  0.987869  2024-01-10
5      1   0.002967  0.997033  2024-01-15
6      1   0.124035  0.875965  2024-01-16
7      1   0.002857  0.997143  2024-01-17
Accuracy: 1.0000
Number of correct answers within the predicted: 4
Number of UP exceeding the threshold 0.5: 4
--------------------------------------------------------------
   Label  Pred_down   Pred_up        Date
0      0   0.986315  0.013685  2024-01-08
1      0   0.991964  0.008036  2024-01-09
3      0   0.935604  0.064396  2024-01-11
4      0   0.948815  0.051185  2024-01-12
8      0   0.511524  0.488476  2024-01-18
9      0   0.564074  0.435926  2024-01-19
Accuracy: 1.0000
Number of correct answers within the predicted: 6
Number of DOWN exceeding the threshold 0.5: 6
--------------------------------------------------------------
   Label  Pred_down   Pred_up        Date
2      1   0.012131  0.987869  2024-01-10
5      1   0.002967  0.997033  2024-01-15
6      1   0.124035 